In [1]:
%load_ext jupyter_black

In [ ]:
import os

os.chdir("..")


In [ ]:
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from excel_lens import ExcelDataExtractor

In [3]:
!pwd

/mnt/c/Users/DELL/OneDrive/Documents/Python/master_scripts/excel-lens


In [5]:
excel_file = ExcelDataExtractor()
curr_file = excel_file.extract("files/dummy_excel_1.xlsx")

In [27]:
df = curr_file.sheets["links_and_images"].tables["table_2"]
df

,Data source links,Description
0,dataset_1,"Lorem ipsum dolor sit amet, consectetur adipis..."
1,dataset_2,"Lorem ipsum dolor sit amet, consectetur adipis..."
2,dataset_3,"Lorem ipsum dolor sit amet, consectetur adipis..."
3,dataset_4,"Lorem ipsum dolor sit amet, consectetur adipis..."
4,dataset_5,"Lorem ipsum dolor sit amet, consectetur adipis..."


In [8]:
tokenizer = AutoTokenizer.from_pretrained(
    "deepseek-ai/deepseek-coder-1.3b-base", trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-coder-1.3b-base",
    torch_dtype=torch.float16,
    trust_remote_code=True,
).cuda()

In [30]:
input_text = """
Given pandas df = curr_file.sheets["links_and_images"].tables["table_2"]
It has a column named "Data source links"
write a code to find unique entries present in that column.
 """
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=128)
code = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(code)

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.



Given pandas df = curr_file.sheets["links_and_images"].tables["table_2"]
It has a column named "Data source links"
write a code to find unique entries present in that column.
 

A: You can use set() to get unique values.
df['Data source links'].unique()




In [ ]:
def extract_code_from_llm_output(text: str) -> list:
    lines = text.splitlines()
    code_lines = []
    for line in lines:
        # Skip LLM notes, markdown, and comments that don't start with code
        if (
            line.strip().startswith("#")
            or "Setting pad_token" in line
            or re.match(r"^\s*$", line)
        ):
            continue
        if re.match(r"^\s*(import|df|pd|\w+\s*=)", line):
            code_lines.append(line)
    return code_lines

In [32]:
cl_code = extract_code_from_llm_output(code)
cl_code

["df['Data source links'].unique()"]

In [ ]:
# Testing model code
exec("import pandas as pd\n" + f"uniq = {cl_code[0]}\nprint(uniq)")

['dataset_1' 'dataset_2' 'dataset_3' 'dataset_4' 'dataset_5']


In [ ]:
# Testing written code
out = exec(
    "import pandas as pd\ndf = pd.read_excel('files/dummy_excel_1.xlsx')\nprint(df['Data source links'].nunique())"
)

5
